# Lesson 05C - Multiple Kernel Size


Most of the code here is based on that [post](file:///C:/Users/ishay/Documents/Data%20Science/Coursera%20and%20practice/ML_Mastery/NLP/Model%20Improvement%20and%20Best%20Practices/How%20to%20Develop%20an%20N-gram%20Multichannel%20Convolutional%20Neural%20Network%20for%20Sentiment%20Analysis.html) from ML Mastery.

In [3]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
#from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from keras.layers import Convolution1D
from keras.layers import Conv1D
from keras.layers import GlobalMaxPooling1D
from keras.layers import MaxPooling1D
from keras.layers import Input
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.merge import concatenate

Using TensorFlow backend.


In [4]:
max_features = 5000

In [5]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features) # instead on nb_words=

**Sequence Padding**

In [6]:
maxlen = 400

We define maximum length of 400. It means that posts shoeter than 400 will be filled with 0s, and longer posts are cut.

In [7]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

**Now defining the model**

In [14]:
def create_model(length, vocab_size):
    # First channel
    inputs_shape1 = Input(shape=(length,))
    embed1 = Embedding(input_dim=vocab_size, output_dim=50)(inputs_shape1)
    spat_drop1 = SpatialDropout1D(0.4)(embed1)
    conv1 = Conv1D(filters=250, kernel_size=3, padding='valid', activation='relu', strides=1)(spat_drop1)
    pool1 = MaxPooling1D(pool_size=2)(conv1)
    flat1 = Flatten()(pool1)
    
    # Second channel
    inputs_shape2 = Input(shape=(length,))
    embed2 = Embedding(input_dim=vocab_size, output_dim=50)(inputs_shape2)
    spat_drop2 = SpatialDropout1D(0.3)(embed2)
    conv2 = Conv1D(filters=250, kernel_size=4, padding='valid', activation='relu', strides=1)(spat_drop2)
    pool2 = MaxPooling1D(pool_size=2)(conv2)
    flat2 = Flatten()(pool2)
    
    # Third channel
    inputs_shape3 = Input(shape=(length,))
    embed3 = Embedding(input_dim=vocab_size, output_dim=50)(inputs_shape3)
    spat_drop3 = SpatialDropout1D(0.4)(embed3)
    conv3 = Conv1D(filters=250, kernel_size=5, padding='valid', activation='relu', strides=1)(spat_drop3)
    pool3 = MaxPooling1D(pool_size=2)(conv3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # FC layers
    dense1 = Dense(250, activation='relu')(merged)
    drop1 = Dropout(0.3)(dense1)
    dense2 = Dense(20, activation='relu')(drop1)
    drop2 = Dropout(0.3)(dense2)
    output = Dense(1, activation='sigmoid')(drop2)
    model = Model(inputs=[inputs_shape1, inputs_shape2, inputs_shape3], outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

In [15]:
model = create_model(maxlen, max_features)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 400, 50)      250000      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_

**Fitting the model**

I must enter X_train three times (also X_test), one for each channel

In [1]:
epochs = 5
batch_size = 32
h = model.fit([X_train, X_train, X_train], y_train, batch_size=batch_size, epochs=epochs, 
              validation_data=([X_test,X_test,X_test], y_test), verbose=1)

NameError: name 'model' is not defined

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
print("Training: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Validation: accuracy1 = %f  ;  loss1 = %f" % (accuracy, loss))

In [ ]:
# A good way to view the shape of each layer. Can be printed after the compile function
print(model.summary())